In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [11]:
IoT = pd.read_csv('../../data/IoT_Garage_Door.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
IoT['state']=encoder.fit_transform(IoT['state'])
IoT['sphone_signal']=encoder.fit_transform(IoT['sphone_signal'])


IoT.head()

/tmp/ipykernel_23231/370847493.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  IoT = pd.read_csv('../../data/IoT_Garage_Door.csv')


,date,time,state,sphone_signal,label,type
0,1-Apr-19,20:53:44,1,1,0,0
1,1-Apr-19,20:53:49,0,0,0,0
2,1-Apr-19,20:53:49,1,1,0,0
3,1-Apr-19,20:53:54,0,0,0,0
4,1-Apr-19,20:53:54,1,1,0,0


In [12]:
x = IoT.drop(['label', 'date', 'time'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [13]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [15]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 4s 54ms/step - loss: 0.4639 - accuracy: 0.9900 - val_loss: 0.2042 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 9ms/step - loss: 0.1449 - accuracy: 1.0000 - val_loss: 0.0943 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 8ms/step - loss: 0.0732 - accuracy: 1.0000 - val_loss: 0.0490 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 7ms/step - loss: 0.0367 - accuracy: 1.0000 - val_loss: 0.0222 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 7ms/step - loss: 0.0163 - accuracy: 1.0000 - val_loss: 0.0092 - val_accuracy: 1.0000


In [16]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

4/4 [==============================] - 1s 2ms/step


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [17]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[0, 0, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )


In [18]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 1ms/step - loss: 0.0092 - accuracy: 1.0000
Test score: 0.009224038571119308
Test accuracy: 1.0
